In [4]:
#@title Imports { form-width: "20%" }
# Some imports that might be useful one day

!pip install recordtype
!pip install ipython-autotime
%load_ext autotime
from recordtype import recordtype
import numpy as np
import urllib.request
import re
import collections
import itertools
import heapq
import functools
import copy

def get_input(day):
  return urllib.request.urlopen(f'https://raw.githubusercontent.com/SantoSimone'
                                f'/Advent-of-Code/master/2020/input_files/input'
                                f'{day}.txt').read().decode('utf-8')

def get_input_as_lines(day):
  return get_input(day).split('\n')[:-1]

def parse_ints(text):
  return [int(x) for x in re.findall(r'\d+', text)]

def splitter(list_to_split, split_val):
  ret = []
  curr = []
  for val in list_to_split:
    if val == split_val:
      ret.append(curr)
      curr = []
      continue
    curr.append(val)
  
  return ret

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 5.87 s (started: 2021-03-07 00:26:27 +00:00)


# Day 1

In [ ]:
#@title Part 1 { form-width: "20%" }

def d1p1():
  expenses = parse_ints(get_input(day))
  for val1, val2 in itertools.combinations(expenses, r=2):
    if val1 + val2 == 2020: return val1, val2 

# 'And so it begins' 
# (for non-nerds: this is a LOTR reference timing 2:46:41 enjoy!)
day = 1
val1, val2 = d1p1()
print(f'The two entries that sum to 2020 are: {val1} and {val2}. Their product'
      f' is {val1*val2}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def d1p2():
  expenses = parse_ints(get_input(day))
  for val1, val2, val3 in itertools.combinations(expenses, r=3):
    if val1 + val2 + val3 == 2020: return val1, val2, val3

val1, val2, val3 = d1p2()
print(f'The three entries that sum to 2020 are: {val1}, {val2} and {val3}. '
      f'Their product is {val1*val2*val3}')

# Day 2

In [ ]:
#@title Part 1 { form-width: "20%" }

def d2p1(inputs):
  def check_policy(line):
    least, most, ch, pw = re.match(r'(\d+)-(\d+) (\w): (\w+)', line).groups()
    return int(least) <= collections.Counter(pw)[ch] <= int(most)

  return sum([check_policy(line) for line in inputs])
  
# It's day 2
day = 2
valid = d2p1(get_input_as_lines(day))
print(f'# of valid passwords: {valid}')

In [ ]:
#@title Part 2 { form-width: "20%" }

# Policy has changed, so we will change our policy func
def d2p2(inputs):
  def check_policy(line):
    least, most, ch, pw = re.match(r'(\d+)-(\d+) (\w): (\w+)', line).groups()
    return (pw[int(least) - 1] == ch) ^ (pw[int(most) - 1] == ch)

  return sum([check_policy(line) for line in inputs])
  

valid = d2p2(get_input_as_lines(day))
print(f'# of valid passwords: {valid}')

# Day 3

In [ ]:
#@title Part 1 { form-width: "20%" }

def d3p1(lines, start, slope):
    def generator(start, slope, width):
        i = 0
        while True:
            yield start[0] + (slope[0] * i) % width, start[1] + slope[1] * i
            i += 1

    def create_map(lines):
        return np.array([
            [x == '#' for x in line]  # columns
            for line in lines  # rows
        ], dtype=np.int32)  # ones and zeros are always nice

    tree_map = create_map(lines)
    height, width = tree_map.shape
    sum = 0
    for c, r in generator(start, slope, width):
        if r >= height: return sum
        sum += tree_map[r, c]

# It's day 3
day = 3
start = (0, 0)
slope = (3, 1) 
trees = d3p1(get_input_as_lines(day), start, slope)
print(f'# of trees with first policy: {trees}')

In [ ]:
#@title Part 2 { form-width: "20%" }

# Not an hard request, we simply need to call previous func 5 times
def d3p2(lines, start, slopes):
  return np.prod([d3p1(lines, start, slope) for slope in slopes])
  
slopes = ((1, 1), (3, 1), (5, 1), (7, 1), (1, 2))
trees = d3p2(get_input_as_lines(day), start, slopes)
print(f'Multiplied values of trees encountered with 5 policies: {trees}')

# Day 4

In [ ]:
#@title Part 1 { form-width: "20%" }
def parse_input(lines):
    ret = []
    passport = {}
    for line in lines:
        if line == '':
            ret.append(passport)
            passport = {}
            continue
        parts = line.split()
        for p in parts:
            k, v = p.split(':')
            passport[k] = v
    ret.append(passport)

    return ret

def check_passport(passport, required_fields):
  return all(field in passport for field in required_fields)

def d4p1(lines):
  passports = parse_input(lines)
  return np.sum([check_passport(passport, required_fields) 
                   for passport in passports])

# It's day 4
day = 4
lines = get_input_as_lines(day)
required_fields = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']
optional_fields = ['cid']
s = d4p1(lines)
print(f'Valid passports: {s}')

In [ ]:
#@title Part 2 { form-width: "20%" }

# Now we want also data validation
def data_validation(passport):
  valid = True
  try:
      num, unit = re.match(r'(\d+)(\w+)', passport['hgt']).groups()
      if unit == 'cm':
          valid = (valid and (150 <= int(num) <= 193))
      elif unit == 'in':
          valid = (valid and (59 <= int(num) <= 76))
      else:
          return False

      return valid and (1920 <= int(passport['byr']) <= 2002) \
              and (2010 <= int(passport['iyr']) <= 2020) \
              and (2020 <= int(passport['eyr']) <= 2030) \
              and passport['hcl'][0] == '#' and len(passport['hcl']) == 7 \
              and all('0' <= c <= '9' or 'a' <= c <= 'f' for c in passport['hcl'][1:]) \
              and passport['ecl'] in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'] \
              and int(passport['pid']) and len(passport['pid']) == 9
  except:
      return False

def d4p2(lines):
  passports = parse_input(lines)
  return np.sum([check_passport(passport, required_fields)
                 and data_validation(passport)
                 for passport in passports])
  

s = d4p2(lines)
print(f'Valid passports with valid data: {s}')

# Day 5

In [ ]:
#@title Part 1 { form-width: "20%" }

def binary_space(line, upper_char, lower_char):
  return int(line.replace(upper_char, '1').replace(lower_char, '0'), base=2)

def d5p1(lines):
  return [binary_space(seat[:7], 'B', 'F') * 8 
          + binary_space(seat[7:], 'R', 'L')
          for seat in lines]

# It's day 5
day = 5
lines = get_input_as_lines(day)
highest = max(d5p1(lines))
print(f'Highest seat ID: {highest}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def check_pre_post(id, ids):
  return id if id + 1 in ids and id - 1 in ids and id not in ids else False

def d5p2(lines):
  ids = d5p1(lines)
  return np.sum([check_pre_post(id, ids) for id in range(min(ids), max(ids))])

my_id = d5p2(lines)
print(f'My ID is: {my_id}')

# Day 6

In [ ]:
#@title Part 1 { form-width: "20%" }

def parse_input(lines):
    return [
            list(g) 
            for k, g in itertools.groupby(lines, lambda x: x != '') if k
           ]

def d6p1(lines):
    groups = parse_input(lines)
    return sum([
        len(set(''.join(group))) for group in groups
    ])

# It's day 5
day = 6
lines = get_input_as_lines(day)
s = d6p1(lines)
print(f'Number of total questions: {s}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def d6p2(lines):
    groups = parse_input(lines)
    all_questions = [set(''.join(group)) for group in groups]
    return sum(
        [
            all(x in g for g in groups[i])
            for i, group_set in enumerate(all_questions)
            for x in group_set
        ]
    )

s = d6p2(lines)
print(f'Number of total questions: {s}')

# Day 7

In [ ]:
#@title Part 1 { form-width: "20%" }

def parse_line(line):
  parts = re.split('bag', line)
  stuff = {
      key.strip(): int(value)
      for p in parts[1:]
      for value, key in re.findall(r'(\d+) ([a-z ]+)', p)
  }
  return {parts[0].strip(): stuff}

def parse_input(lines):
  bags = {}
  for line in lines:
      bags.update(parse_line(line))
  return bags

def rec_contains(bags, bag, bag_to_place):
  return bag_to_place == bag \
          or any(rec_contains(bags, inside_bag, bag_to_place) 
                 for inside_bag in bags[bag])

def d7p1(lines, bag_to_place):
  bags = parse_input(lines)
  return [rec_contains(bags, bag, bag_to_place) for bag in bags]

# It's day 7
day = 7
lines = get_input_as_lines(day)
# We sub 1 because shiny gold is counted in our algorithm
s = sum(d7p1(lines, 'shiny gold')) - 1
print(f'Shiny gold bag is contained in : {s}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def count_bags_inside(bags, outer_bag):
  # When we reach an empty bag we count 1 and we always add 1 to the count 
  # of inner bags cause we also count current outer bag
  return 1 + sum([num * count_bags_inside(bags, inner) 
                  for inner, num in bags[outer_bag].items()])


def d7p2(lines, bag_to_place):
  return count_bags_inside(parse_input(lines), bag_to_place)

# We sub 1 because shiny gold is counted in our algorithm
s = d7p2(lines, 'shiny gold') - 1
print(f'Shiny gold contains {s} bags')

# Day 8

In [ ]:
#@title Part 1 { form-width: "20%" }

Instruction = collections.namedtuple('Instruction', ['code', 'value'])

def parse_instructions(lines):
  return [
          Instruction(code=inst, value=int(value)) 
          for inst, value in [line.split() for line in lines]
          ]

def execute(inst, idx, acc):
  if inst.code == 'acc': return idx + 1, acc + inst.value
  elif inst.code == 'jmp': return idx + inst.value, acc
  elif inst.code == 'nop': return idx + 1, acc

def d8p1(instructions):
  i, accumulator, done = 0, 0, set()
  while i not in done:
    done = done.union({i})
    i, accumulator = execute(instructions[i], i, accumulator)

  return accumulator

# It's day 8
day = 8
instructions = parse_instructions(get_input_as_lines(day))
acc = d8p1(instructions)
print(f'Right before repeating an instruction the accumulator is: {acc}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def changed_instructions(instructions):
  changes = {'nop': 'jmp', 'jmp': 'nop'}
  for i, inst in enumerate(instructions):
    if inst.code in changes.keys():
      yield instructions[:i] \
            + [Instruction(code=changes[inst.code], value=inst.value)] \
            + instructions[i+1:] 

def d8p2(instructions):
  for changed in changed_instructions(instructions):
      i, accumulator, done = 0, 0, set()
      while i <= len(changed) - 1 and i not in done:
          done = done.union({i})
          i, accumulator = execute(changed[i], i, accumulator)
      if i >= len(changed): return accumulator

instructions = parse_instructions(get_input_as_lines(day))
acc = d8p2(instructions)
print(f'Changing one single operation the program terminates with: {acc}')

# Day 9

In [ ]:
#@title Part 1 { form-width: "20%" }

def valid(next, previous):
  prev_set = set(previous)
  if next not in map(lambda x: x[0] + x[1], itertools.combinations(prev_set, 2)):
    return False
  return True

def d9p1(nums, preamble):
    for i, num in enumerate(nums[preamble:]):
        if not valid(num, nums[i:i+preamble]): return num
    return -np.inf

# It's day 9
day = 9
nums = parse_ints(get_input(day))
invalid = d9p1(nums, 25)
print(f'First number that does not respect the policy is: {invalid}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def d9p2(nums, preamble):
  invalid = d9p1(nums, preamble)
  start, end, partial = 0, 0, 0
  while partial != invalid:
      if partial > invalid:
          partial -= nums[start]
          start += 1
          continue
      else:
          partial += nums[end]
      end += 1
  return min(nums[start:end]) + max(nums[start:end])

weakness = d9p2(nums, 25)
print(f'The encryption weakness is: {weakness}')

# Day 10

In [ ]:
#@title Part 1 { form-width: "20%" }

def jolt_diff(low, high, condition):
  return 1 if (high - low) == condition else 0

def d10p1(adapters):
  sorted_adapters = sorted(adapters)
  subsequent = [(0, sorted_adapters[0])]  # first connection
  subsequent += [
                 (sorted_adapters[i], sorted_adapters[i + 1])
                 for i, _ in enumerate(sorted_adapters[:-1])
                ]
  ones = sum(map(lambda x: jolt_diff(x[0], x[1], 1), subsequent))
  threes = sum(map(lambda x: jolt_diff(x[0], x[1], 3), subsequent)) + 1
  return ones * threes

# It's day 10
day = 10
adapters = parse_ints(get_input(day))
ret = d10p1(adapters)
print(f'Multiplication of 1-jolts by 3-jolts is: {ret}')

In [ ]:
#@title Part 2 { form-width: "20%" }
# This was a though one: did not understand the stack of calls could be so
# overwhelming in a brute-force approach..
# Had to switch from a for-loop (A* search) to recursion (lru_cache friendly) 

State = collections.namedtuple('State', 'jolts, adapters')

def d10p2(adapters):
    # Add last connection
    adapters = sorted(adapters)
    goal = max(adapters) + 3  # final connection is built-in adapter
    adapters.append(goal)
    return recursive_search(State(0, frozenset(adapters)), goal)


@functools.lru_cache(maxsize=500)
def recursive_search(curr, goal):
    if curr.jolts == goal: return 1
    possible_nexts = possible_connections(curr)
    if not possible_nexts: return 0
    return sum([
        recursive_search(next_adap, goal) for next_adap in possible_nexts
    ])


def possible_connections(state):
    possibles = filter(lambda x: state.jolts < x <= state.jolts + 3, state.adapters)
    return [State(adap, state.adapters.difference({a for a in state.adapters if a <= adap})) for adap in possibles]

adapters = parse_ints(get_input(day))
ret = d10p2(adapters)
print(f'Number of possible configurations is: {ret}')

# Day 11

In [ ]:
#@title Part 1 { form-width: "20%" }

def create_seats_map(lines):
  # Seats will be: -1 floor, 0 empty, 1 occupied
  seats = [line.replace('.', '0').replace('L', '1') for line in lines]
  seats = [[int(s) for s in line] for line in seats]
  seats = np.array(seats) - 1
  return seats

def num_adjacent_occupied(seats, r, c):
  rows = [max(0, r - 1), r, min(r + 1, seats.shape[0] - 1)]
  cols = [max(0, c - 1), c, min(c + 1, seats.shape[1] - 1)]
  adjacent = set(itertools.product(rows, cols))
  # We do not count our seat, so we remove it from combinations
  adjacent.remove((r,c))
  return sum([1 for row, col in adjacent if seats[row, col] == 1])

def round_seat(seats, max_adjacent):
  def evaluate_single(seat, r, c):
    if seat == -1: return -1
    num_adjacent = num_adjacent_occupied(seats, r, c)
    
    # Empty and no occupied adjacent -> occupied
    if seat == 0 and num_adjacent == 0: return 1
    elif seat == 0: return 0
    
    # Occupied and `max_adjacent` or more adjacent occupied -> empty
    if seat == 1 and num_adjacent >= max_adjacent: return 0
    else: return 1

  new_seats = np.array(
      [evaluate_single(s, r, c)
       for (r, c), s in np.ndenumerate(seats)
      ]
  )
  # Above operation outputs the flattened matrix, so we reshape
  return np.reshape(new_seats, (seats.shape[0], seats.shape[1]))


def d11p1(lines, max_adjacent):
  last_seats = create_seats_map(lines)
  while True:
    new_seats = round_seat(last_seats, max_adjacent)
    if all((new_seats == last_seats).reshape(-1)):
      return sum(new_seats[new_seats > 0])
    last_seats = new_seats[:, :]

# It's day 11
day = 11
lines = get_input_as_lines(day)
ret = d11p1(lines, 4)
print(f'Final number of occupied seats is: {ret}')

In [ ]:
#@title Part 2 { form-width: "20%" }

# We only need to change the policies
def num_adjacent_occupied(seats, r, c):
    def check_direction(r, c, direction):
        row_dx, col_dx = direction
        h, w = seats.shape
        for i in range(1, max(h, w)):
            if not (-1 < r + row_dx * i < h and -1 < c + col_dx * i < w): return 0
            if seats[r + row_dx * i, c + col_dx * i] == -1:
                continue
            else:
                return seats[r + row_dx * i, c + col_dx * i]
        return 0

    directions = [(-1, -1), (-1, 0), (-1, 1),  # Topleft - Top - Topright
                  (0, -1), (0, 1),  # Left - Right
                  (1, -1), (1, 0), (1, 1)]  # Bottomleft - Bottom - Bottomright

    return sum([check_direction(r, c, dir) for dir in directions])

# It's day 11
day = 11
lines = get_input_as_lines(day)
ret = d11p1(lines, 5)
print(f'Final number of occupied seats is: {ret}')

# Day 12

In [ ]:
#@title Part 1 { form-width: "20%" }

def parse_instructions(input_txt):
    return [(code, int(value)) for (code, value) in re.findall(r'(\w)(\d+)', input_txt)]

def d12p1(input_txt):
    instructions = parse_instructions(input_txt)
    # East, North, West, South
    directions = np.array([(1, 0), (0, 1), (-1, 0), (0, -1)])
    direction_idx = 0
    pos = np.array([0, 0])
    for code, value in instructions:
        if code == 'N': pos += (0, value)
        if code == 'S': pos += (0, -value)
        if code == 'E': pos += (value, 0)
        if code == 'W': pos += (-value, 0)
        if code == 'L': direction_idx += (value // 90)
        if code == 'R': direction_idx -= (value // 90)
        if code == 'F': pos += directions[direction_idx % 4] * value

    return sum(np.abs(pos))

# It's day 12
day = 12
input_txt = get_input(day)
ret = d12p1(input_txt)
print(f'Manhattan distance from starting position (0,0) is: {ret}')

In [ ]:
#@title Part 2 { form-width: "20%" }
# Wanted to give a little bit of structure to ease the read of rotation
Position = recordtype('Position', 'x y')

def d12p2(input_txt):
  instructions = parse_instructions(input_txt)
  waypoint_pos = Position(x=10, y=1)
  ship_pos = Position(x=0, y=0)
  for code, value in instructions:
    if code == 'N': waypoint_pos.y += value
    elif code == 'S': waypoint_pos.y -= value
    elif code == 'E': waypoint_pos.x += value
    elif code == 'W': waypoint_pos.x -= value
    elif code == 'F':
        ship_pos.x += waypoint_pos.x * value
        ship_pos.y += waypoint_pos.y * value
    else:  # code is L or R now
      value = -value if code == 'L' else value
      # Cannot format decently, so i will keep this oneliners
      cos_v, sin_v = int(np.cos(np.deg2rad(value))), int(np.sin(np.deg2rad(value)))
      waypoint_pos.x, waypoint_pos.y = waypoint_pos.x * cos_v + waypoint_pos.y * sin_v, -waypoint_pos.x * sin_v + waypoint_pos.y * cos_v

  return np.abs(ship_pos.x) + np.abs(ship_pos.y)

ret = d12p2(input_txt)
print(f'Manhattan distance from starting position (0,0) is: {ret}')

# Day 13

In [ ]:
#@title Part 1 { form-width: "20%" }

def d13p1(input_txt):
  inputs = parse_ints(input_txt)
  start, buses = inputs[0], np.array(inputs[1:])
  first_bus_after_timestamp = ((start // buses) + 1) * buses
  bus_id = buses[np.argmin(first_bus_after_timestamp)]
  minutes_to_wait = np.min(first_bus_after_timestamp) - start
  return bus_id * minutes_to_wait

# It's day 13
day = 13
input_txt = get_input(day)
ret = d13p1(input_txt)
print(f'Bus ID times minutes to wait is: {ret}')

In [ ]:
#@title Part 2 { form-width: "20%" }

# We need to parse differently now

def parse_inputs(buses_line):
    buses = parse_ints(buses_line)
    subsequents = np.array(
        [-i for i, ts in enumerate(buses_line.split(',')) if ts != 'x']
    )
    return buses, subsequents

# Colab runs Pyton 3.6 so we need to run euclidean algorithm instead of an easy
# oneliner -> s_i = pow(big_n_i, -1, bus_id)
def euclidean_algo(a, b):
    old_r, r = a, b
    old_s, s = 1, 0
    while r != 0:
        q = old_r // r
        old_r, r = r, old_r % r
        old_s, s = s, old_s - q * s
    return old_s


def d13p2(lines):
    buses, subsequents = parse_inputs(lines[1])
    
    # Chinese Remainder Theorem saves us lots of time, THANKS!
    big_n = np.prod(buses)
    big_n_i = big_n // buses
    s_i = [euclidean_algo(n, id) for id, n in zip(buses, big_n_i)]
    timestamp = sum(
        [offset * s * n for offset, s, n in zip(subsequents, s_i, big_n_i)]
    )
    return timestamp % big_n

lines = get_input_as_lines(day)
ret = d13p2(lines)
print(f'First timestamp that meets the condition is: {ret}')

# Day 14

In [ ]:
#@title Part 1 { form-width: "20%" }

# This day's code will probably be refactored one day, i am pretty tired and i
# should go to bed, anyway let's end this quickly

def write_on_mem(mem, address, value, mask):
  value = f'{value:036b}'
  value = [mask[i] if mask[i] != 'X' else value[i] for i in range(len(mask))]
  mem[address] = int(''.join(value), 2)

def d14p1(lines):
  mem = {}
  for line in lines:
    if line.startswith('mask'):
      mask = line.split('=')[1].strip()
    elif line.startswith('mem'):
      address, value = parse_ints(line)
      write_on_mem(mem, address, value, mask)
  
  return sum(mem.values())

# It's day 14
day = 14
lines = get_input_as_lines(day)
ret = d14p1(lines)
print(f'Sum of all values left in memory is: {ret}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def write_on_mem(mem, address, value, mask):
  def actual_write(floatings):
    floatings = list(reversed(floatings))
    write_address = []
    for i, mask_val in enumerate(mask):
      if mask_val == '1': write_address.append(mask_val)
      elif mask_val == 'X': write_address.append(floatings.pop())
      else: write_address.append(address[i])
    write_address = int(''.join(write_address), 2)
    mem[write_address] = value

  address = f'{address:036b}'
  masked_address = [mask[i] if mask[i] != '0' else address[i] 
                    for i in range(len(mask))]
  x_count = masked_address.count('X')
  for fl in [f'{x:0{x_count}b}' for x in range(2**x_count)]:
    actual_write(fl)

ret = d14p1(lines)
print(f'Sum of all values left in memory is: {ret}')

# Day 15

In [ ]:
#@title Part 1 { form-width: "20%" }

def d15p1(input_nums):
  last_num = input_nums[-1]
  for i in range(len(input_nums), 2020):
    last_num = input_nums[-1]
    if last_num not in input_nums[:-1]:
      input_nums.append(0)
    else:
      input_nums.append(
        # last occurence - previous occurence
        len(input_nums) - (len(input_nums) - 1 - input_nums[:-1][::-1].index(last_num))
      )

  return input_nums[-1]

# It's day 15, today input is not from some file
input_nums = [8, 13, 1, 0, 18, 9]
ret = d15p1(input_nums)
print(f'2020th number spoken is: {ret}')

In [ ]:
#@title Part 2 { form-width: "20%" }

# Obviously list were the easy way, let's implemented the faster way
def d15p2(input_nums):
  spoken = {val: i + 1 for i, val in enumerate(input_nums)}
  num = 0
  for i in range(len(input_nums) + 1, 30000000):
    if num in spoken:
      spoken[num], num = i, i - spoken[num]
    else:
      spoken[num], num = i, 0

  return num

# It's day 15, today input is not from some file
input_nums = [8, 13, 1, 0, 18, 9]
ret = d15p2(input_nums)
print(f'30.000.000-th number spoken is: {ret}')

# Day 16

In [ ]:
#@title Part 1 { form-width: "20%" }

def parse_rule(line):
  k = line.split(':')[0]
  v = []
  for r1, r2 in re.findall(r'(\d+)-(\d+)', line):
    v = itertools.chain(v, range(int(r1), int(r2) + 1))
  return (k, list(v))

def parse_inputs(lines):
  # rules - ticket - nearby are divided by blank line
  sep = lines.index('')
  rules_lines = lines[:sep]
  my_ticket = lines[sep + 2]  # line above stating 'your ticket' is useless
  nearby = lines[sep + 5:]  # same as above
  rules = [parse_rule(r) for r in rules_lines]

  my_ticket = parse_ints(my_ticket)
  nearbies = [parse_ints(n) for n in nearby]
  return rules, my_ticket, nearbies

def scanning_error(ticket, rules):
  return sum(
      [val for val in ticket 
      if all(val not in r for r in rules)]
  )

def d16p1(lines):
  rules, my_ticket, nearbies = parse_inputs(lines)
  rules_values = [r[1] for r in rules]
  return sum([scanning_error(t, rules_values) for t in nearbies])


# It's day 16
day = 16
lines = get_input_as_lines(day)
ret = d16p1(lines)
print(f'Number of invalid tickets is: {ret}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def possible_columns(rules, valid_tickets):
  ret = []
  for rule in rules:
    valid = []
    for i in range(len(rules)):
      if sum([scanning_error([t[i]], [rule[1]]) for t in valid_tickets]) == 0:
        valid.append(i)
    ret.append(valid)
  return ret

# Gotta admit this was a bit hard, so i preferred saving some time and get
# from StackOverflow
def product_without_duplicates(*cols):
  def inner(i):
    if i == n:
      yield tuple(result)
      return
    for elt in sets[i] - seen:
      seen.add(elt)
      result[i] = elt
      for t in inner(i+1):
        yield t
      seen.remove(elt)

  sets = [set(seq) for seq in cols]
  n = len(sets)
  seen = set()
  result = [None] * n
  for t in inner(0):
    yield t

def d16p2(lines):
  rules, my_ticket, nearbies = parse_inputs(lines)
  rules_values = [r[1] for r in rules]
  valid_tickets = [t for t in nearbies if scanning_error(t, rules_values) == 0]
  cols = possible_columns(rules, valid_tickets)
  for perm in product_without_duplicates(*cols):
    rules_values = [rules[r][1] for r in perm]
    if sum([scanning_error(t, rules_values) for t in valid_tickets]) == 0:
      return np.prod([my_ticket[i] for i, p in enumerate(perm)
                      if rules[i][0].startswith('departure')])

ret = d16p2(lines)
print(f'Multiplication of desired values in my ticket is: {ret}')

# Day 17

In [ ]:
#@title Part 1 { form-width: "20%" }

def parse_input_grid(lines):
    grid = [
        [1 if x == '#' else 0 for x in line]
        for line in lines
    ]
    return np.array(grid)


def neighbours(pos):
    possible_moves = list(itertools.product([-1, 0, 1], repeat=len(pos)))
    # We are not neighbours of ourselves
    possible_moves.remove((0,) * len(pos))
    for s in possible_moves:
        yield tuple(np.array(pos) + np.array(s))


def round_op(cubes):
  def check_cube(pos):
    active_neighbors = sum([cubes[p]
                            for p in neighbours(pos)
                            if all([0 <= dim < cubes.shape[i] 
                                    for i, dim in enumerate(p)])
                           ])
    if cubes[pos]:
      return 1 if active_neighbors in [2, 3] else 0
    else:
      return 1 if active_neighbors == 3 else 0

  return np.array([check_cube(pos) 
                  for pos, _ in np.ndenumerate(cubes)]
                  ).reshape(cubes.shape)

def d17p1(lines, rounds):
  cubes = parse_input_grid(lines)
  cubes = np.expand_dims(cubes, -1)
  for i in range(rounds):
    cubes = np.pad(cubes, 1)
    cubes = round_op(cubes)
  return np.sum(cubes)

# It's day 17
day = 17
lines = get_input_as_lines(day)
ret = d17p1(lines, 6)
print(f'After 6 rounds, number of active cubes is {ret}')

In [ ]:
#@title Part 2 { form-width: "20%" }

# This takes a little bit of time (around 1m 30s), should investigate how to 
# optimize that
def d17p2(lines, rounds):
  cubes = parse_input_grid(lines)
  cubes = np.expand_dims(np.expand_dims(cubes, -1), -1)
  for i in range(rounds):
    cubes = np.pad(cubes, 1)
    cubes = round_op(cubes)
  return np.sum(cubes)

ret = d17p2(lines, 6)
print(f'After 6 rounds, number of active cubes is {ret}')

# Day 18

In [ ]:
#@title Part 1 { form-width: "20%" }

def solve_simple(line):
    line = line.replace('(', '').replace(')', '')
    ret = 0
    next_op = '+'
    if len(line) < 2:
        return ret
    for c in re.split(r'(\+|\*)', line):
        if c == '+' or c == '*':
            next_op = c
        elif c.strip().isdigit():
            ret = ret + int(c) if next_op == '+' else ret * int(c)

    return ret


def resolve(line):
    while len(re.findall(r'(\([0-9+* ]+\))', line)) > 0:
        for par in re.findall(r'(\([0-9+* ]+\))', line):
            line = line.replace(par, str(solve_simple(par)))

    return solve_simple(line)


def d18p1(lines):
    return sum([resolve(line) for line in lines])

# It's day 18
day = 18
lines = get_input_as_lines(day)
ret = d18p1(lines)
print(f'Sum of results is: {ret}')

Sum of results is: 69490582260
time: 810 ms (started: 2020-12-22 12:14:46 +00:00)


In [ ]:
#@title Part 2 { form-width: "20%" }

# We need to solve plus first, then muls
# Little changes to consider overflows

def solve_simple(line):
    while len(re.findall(r'(\d+ \+ \d+)+', line)) > 0:
        for plus in re.findall(r'(\d+ \+ \d+)+', line)[:1]:
            line = line.replace(plus, str(sum(parse_ints(plus))))

    return np.prod(np.longlong(parse_ints(line)))

def resolve(line):
    while len(re.findall(r'(\([0-9+* ]+\))', line)) > 0:
        for par in re.findall(r'(\([0-9+* ]+\))', line):
            line = line.replace(par, str(solve_simple(par)))

    return np.longlong(solve_simple(line))

ret = d18p1(lines)
print(f'Sum of results is: {ret}')

Sum of results is: 362464596624526
time: 49.3 ms (started: 2020-12-22 12:40:54 +00:00)


# Day 19

In [ ]:
#@title Part 1 { form-width: "20%" }

# I will highly leverage the re package, after all this day is just a remake of
# regex matching

def process_rule(rules, number):
  ret = "("
  rule = rules[number].split()

  for val in rule:
    if val.isnumeric():
      ret += process_rule(rules, val)
    else:
      ret += val

  return ret + ")"

def d19p1(lines):
  rules, messages = lines[:lines.index('')], lines[lines.index('') + 1:]
  rules_dict = {
      rule.split(": ")[0]: rule.split(": ")[1].replace('"','')
      for rule in rules
  }

  regex = re.compile(process_rule(rules_dict, "0"))
  ret = [1 if re.fullmatch(regex, msg) else 0 for msg in messages]
  return sum(ret)

# It's day 19
day = 19
lines = get_input_as_lines(day)
ret = d19p1(lines)
print(f'Number of messages that match rule 0 is: {ret}')

Number of messages that match rule 0 is: 224
time: 85.5 ms (started: 2021-02-22 21:53:28 +00:00)


In [ ]:
#@title Part 2 { form-width: "20%" }
def d19p2(lines):
  rules, messages = lines[:lines.index('')], lines[lines.index('') + 1:]
  rules_dict = {
      rule.split(": ")[0]: rule.split(": ")[1].replace('"','')
      for rule in rules
  }
  # hard coded for brevity
  rules_dict["8"] = "42 +"
  rules_dict["11"] = "42 ( 42 ( 42 ( 42 ( 42 ( 42 ( 42 ( 42 ( 42 ( 42 ( 42 31 )? 31 )? 31 )? 31 )? 31 )? 31 )? 31 )? 31 )? 31 )? 31 )? 31"
  regex = re.compile(process_rule(rules_dict, "0"))
  ret = [1 if re.fullmatch(regex, msg) else 0 for msg in messages]
  return sum(ret)

# It's day 19
day = 19
lines = get_input_as_lines(day)
ret = d19p2(lines)
print(f'Number of messages that match rule 0 is: {ret}')

Number of messages that match rule 0 is: 436
time: 625 ms (started: 2021-02-22 22:38:03 +00:00)


# Day 20

In [6]:
#@title Part 1 { form-width: "20%" }

def parse_inputs(lines):
  def parse_tile(tile_block):
    uid = int(re.findall(r'([\d]+)', tile_block[0])[0])
    square_dim = len(tile_block[1:])
    tiles[uid] = np.array(
        [1 if x == '#' else 0 for line in tile_block[1:] for x in line]
    ).reshape((square_dim, square_dim))

  tiles = {}
  for tile in splitter(lines, ''):
    parse_tile(tile)

  return tiles


def edges(tile):
  return list(map(tuple, [tile[0], tile[:, -1], tile[-1], tile[:, 0]]))

def top_edge(tile): return tile[0]
def left_edge(tile): return tile[:, 0]
def bottom_edge(tile): return tile[-1]
def right_edge(tile): return tile[:, -1]

def flip(tile):
  return np.flip(tile, 0)


def rot90(tile):
  return np.rot90(tile)


def transform(tile):
  ret = [tile]
  tmp = copy.deepcopy(tile)
  for _ in range(3):
    ret.append(rot90(tmp))
    tmp = rot90(tmp)
  ret += [flip(t) for t in ret]
  return ret


def d20p1(lines):
  tiles = parse_inputs(lines)
  all_transforms = {k: transform(v) for k, v in tiles.items()}
  possible_edges = {}
  for uid, tile in tiles.items():
    possible_edges[uid] = set.union(*[set(edges(t)) for t in transform(tile)])

  dim = int(len(tiles) ** 0.5)
  grid = np.array([[(None, None)] * dim for _ in range(dim)])
  used = set()

  possible_neighbor = collections.defaultdict(list)
  for i, j in itertools.combinations(tiles.keys(), 2):
    if len(possible_edges[i] & possible_edges[j]) == 2:
      possible_neighbor[i].append(j)
      possible_neighbor[j].append(i)

  def recursion(pos):
    row, col = np.divmod(pos, dim)
    if pos == dim**2:
      return True
    if pos == 0:
      neighbors = tiles.keys()
    else:
      neighbors = set()
      if row > 0:
        neighbors = neighbors.union(set(possible_neighbor[grid[row - 1, col][0]]))
      if col > 0:
        neighbors = neighbors.union(set(possible_neighbor[grid[row, col - 1][0]]))
    for tile_id in neighbors:
      if tile_id in used:
        continue
      for i, t in enumerate(all_transforms[tile_id]):
        top, left = True, True
        if row > 0:
          top_id, top_transform = grid[row - 1, col]
          top_tile = all_transforms[top_id][top_transform]
          top = all(bottom_edge(top_tile) == top_edge(t))
        if col > 0:
          left_id, left_transform = grid[row, col - 1]
          left_tile = all_transforms[left_id][left_transform]
          left = all(right_edge(left_tile) == left_edge(t))
        if top and left:
          grid[row, col] = tile_id, i
          used.add(tile_id)
          if recursion(pos + 1):
            return True
          used.remove(tile_id)
    return False

  recursion(0)

  corners = [grid[r, c][0] for r, c in [(0, 0), (0, dim-1), (dim-1, 0), (dim-1, dim-1)]]

  return corners, grid



# It's day 20
day = 20
lines = get_input_as_lines(day)
ret, _ = d20p1(lines)
print(f'Product of 4 corners is: {np.prod(ret, dtype=np.longlong)}')

Product of 4 corners is: 7901522557967
time: 413 ms (started: 2021-03-07 00:26:47 +00:00)


In [7]:
#@title Part 2 { form-width: "20%" }

def d20p2(lines):
  _, grid = d20p1(lines)
  tiles = parse_inputs(lines)
  all_transforms = {k: transform(v) for k, v in tiles.items()}
  image = np.concatenate(
    [
      np.concatenate(
        [all_transforms[grid[i, j][0]][grid[i, j][1]][1:-1, 1:-1]
         for j in range(12)],
        axis=1
      )
      for i in range(12)
    ],
    axis=0
  )

  string = '                  # \n#    ##    ##    ###\n #  #  #  #  #  #   '
  sea_monster = np.array(
      [
          [1 if c == '#' else 0 for c in line]
          for line in string.splitlines()
      ]
  )

  count = 0
  for img in transform(image):
    for i in range(img.shape[0] - sea_monster.shape[0] + 1):
      for j in range(img.shape[1] - sea_monster.shape[1] + 1):
        check = img[i:i+sea_monster.shape[0], j:j+sea_monster.shape[1]]
        if ((check & sea_monster) == sea_monster).all():
          count += 1
    if count: break

  return np.sum(image) - count * np.sum(sea_monster)

# It's day 20
day = 20
lines = get_input_as_lines(day)
ret = d20p2(lines)
print(f'Number of \'#\' not in sea monsters is: {ret}')

Number of '#' not in sea monsters is: 2476
time: 609 ms (started: 2021-03-07 00:30:05 +00:00)


# Day 21

In [ ]:
#@title Part 1 { form-width: "20%" }

def d21p1(lines):
  def split_ingredients_allergens(line):
    ingredients, allergens = line.split('(')
    ingredients = re.findall(r'(\w+)+', ingredients)
    allergens = re.findall(r'(\w+)+', allergens)[1:]
    return set(ingredients), set(allergens)

  allergens_dict = {}
  ing_count = collections.defaultdict(int)
  for line in lines:
    ingredients, allergens = split_ingredients_allergens(line)
    for all in allergens:
      if all in allergens_dict:
        allergens_dict[all].intersection_update(ingredients)
      else:
        allergens_dict[all] = set(ingredients)

    for ing in ingredients:
      ing_count[ing] += 1

  ret = sum(
      [ing_count[x] for x in ing_count if not any(x in a for a in allergens_dict.values())]
  )

  return ret, allergens_dict


# It's day 21
day = 21
lines = get_input_as_lines(day)
ret, a = d21p1(lines)
print(f'Ingredients that cannot possibly contain any of the allergens appear {ret} times')

Ingredients that cannot possibly contain any of the allergens appear 2211 times
time: 160 ms (started: 2021-02-27 08:08:37 +00:00)


In [ ]:
#@title Part 2 { form-width: "20%" }

def d21p2(lines):
  def update_dict(ingredient):
    for a in allergens_dict:
      if ingredient in allergens_dict[a]:
        allergens_dict[a].discard(ingredient)

  allergens_dict = d21p1(lines)[1]
  tot = len(allergens_dict)
  identified = []
  while len(identified) < tot:
    for a in allergens_dict:
      if len(allergens_dict[a]) == 1:
        ing = allergens_dict[a].pop()
        identified.append((a, ing))
        update_dict(ing)
  
  identified = [x[1] for x in sorted(identified, key=lambda x: x[0])]

  return ','.join(identified)


# It's day 21
day = 21
lines = get_input_as_lines(day)
ret = d21p2(lines)
print(f'My canonical dangerous ingredient list is: {ret}')

My canonical dangerous ingredient list is: vv,nlxsmb,rnbhjk,bvnkk,ttxvphb,qmkz,trmzkcfg,jpvz
time: 83.1 ms (started: 2021-02-26 18:58:06 +00:00)


# Day 22

In [ ]:
#@title Part 1 { form-width: "20%" }

def get_decks(lines):
  deck1 = [int(n) for n in lines[:lines.index('')] if n.isdigit()]
  deck2 = [int(n) for n in lines[lines.index(''):] if n.isdigit()]
  return deck1, deck2

def d22p1(lines):
  p1, p2 = get_decks(lines)
  _, ret = play_game(p1, p2)
  return ret

def proc_round(p1, p2):
  p1_card = p1.pop(0)
  p2_card = p2.pop(0)
  if p1_card > p2_card:
    p1.extend([p1_card, p2_card])
  else:
    p2.extend([p2_card, p1_card])
  
def play_game(p1, p2):
  while len(p1) > 0 and len(p2) > 0:
    proc_round(p1, p2)

  ret_v = p1 if len(p1) > 0 else p2
  ret_p = 0 if len(p1) > 0 else 1
  return ret_p, np.sum(
      np.array(ret_v) * np.array(list(reversed(range(1, len(ret_v)+1))))
  )


# It's day 22
day = 22
lines = get_input_as_lines(day)
ret = d22p1(lines)
print(f'Winning player\'s score is: {ret}')

Winning player's score is: 34664
time: 168 ms (started: 2021-02-27 09:30:31 +00:00)


In [ ]:
#@title Part 2 { form-width: "20%" }

# We just need to change the round processing to get some recursion
# And to add the infinite loop check
def proc_round(p1, p2, played1, played2):
  if tuple(p1) in played1 or tuple(p2) in played2:
    return None
  played1.add(tuple(p1))
  played2.add(tuple(p2))

  p1_card = p1.pop(0)
  p2_card = p2.pop(0)

  winner = 0 if p1_card > p2_card else 1
  if len(p1) >= p1_card and len(p2) >= p2_card:
    winner, _ = play_game(p1[:p1_card], p2[:p2_card])

  if winner == 0:
    p1.extend([p1_card, p2_card])
  else:  # winner == 1
    p2.extend([p2_card, p1_card])

  return winner

def play_game(p1, p2):
  played1, played2 = set(), set()
  while len(p1) > 0 and len(p2) > 0:
    now = proc_round(p1, p2, played1, played2)
    if now is None:
      p2 = []
      break

  ret_v = p1 if len(p1) > 0 else p2
  ret_p = 0 if len(p1) > 0 else 1
  return ret_p, np.sum(
    np.array(ret_v) * np.array(list(reversed(range(1, len(ret_v) + 1))))
  )


ret = d22p1(lines)
print(f'Winning player\'s score is: {ret}')

Winning player's score is: 32018
time: 1.16 s (started: 2021-02-27 09:45:49 +00:00)


# Day 23

In [ ]:
#@title Part 1 { form-width: "20%" }

# Needed a while to implement all this, so i want to document a bit.
# Since part 2 asks for an efficient implementation, instead of using an array
# that gets rolled many times, I'm using a 'successor' array where the i-th 
# position means that the successor of value i is successor[i]. This way only 3
# values get swapped every loop.
# To be cleaner, also the destination check has been more efficient
# Result is not "efficient", but 15s for part 2 is way better than initial
# 4 days required by quick and dirty implementation :)

def d23p1(cups, moves):
  def get_next(pos, n):
    r = []
    for i in range(n):
      r.append(succ[pos])
      pos = succ[pos]
    return r

  succ = [None] * (len(cups) + 1)
  for i, x in enumerate(cups):
    succ[x] = cups[(i+1) % len(cups)]

  curr_pos = cups[0]
  for _ in range(moves):
    three = get_next(curr_pos, 3)

    dst = curr_pos - 1
    while True:
      if dst < 1:
        dst = max(cups)
      if dst not in three:
        break
      dst -= 1

    succ[curr_pos], succ[dst], succ[three[-1]] = succ[three[-1]], succ[curr_pos], succ[dst]

    curr_pos = succ[curr_pos]

  return get_next(1, 8)



# It's day 23.. but no input from the net
cups = list(map(int, '394618527'))
ret = d23p1(cups, 100)
ret = ''.join(map(str, ret))
print(f'Final label order: {ret}')

Final label order: 78569234
time: 41.3 ms (started: 2021-02-27 18:13:47 +00:00)


In [ ]:
#@title Part 2 { form-width: "20%" }

cups = list(map(int, '394618527'))
max_num = max(cups)
for i in range(max_num + 1, 1_000_001):
  cups.append(i)
ret = d23p1(cups, 10_000_000)
ret = ret[0] * ret[1]
print(f'Final label order: {ret}')

Final label order: 565615814504
time: 15.7 s (started: 2021-02-27 18:14:24 +00:00)


# Day 24

In [ ]:
#@title Part 1 { form-width: "20%" }

moves = {
  'w': (0, -2),
  'sw': (1, -1),
  'se': (1, 1),
  'e': (0, 2),
  'ne': (-1, 1),
  'nw': (-1, -1)
}

def process_grid(lines, size):
  hex_grid = np.zeros((size, size))
  for i in range(size):
    for j in range(size):
      if (i + j) % 2 == 0:
        hex_grid[i, j] = 1

  def parse_steps(line):
    steps = []
    window = 1
    i = 0
    while i < len(line):
      if line[i:i + window] in moves.keys():
        steps.append(moves[line[i:i + window]])
        i += window
        window = 1
      else:
        window += 1

    return steps

  ref_x = ref_y = size // 2
  for line in lines:
    steps = parse_steps(line)
    y = sum([x[0] for x in steps])
    x = sum([x[1] for x in steps])
    hex_grid[ref_y + y, ref_x + x] *= -1
  
  return hex_grid

def d24p1(lines, size):
  hex_grid = process_grid(lines, size)
  counts = collections.Counter(hex_grid.ravel())
  return counts[-1] if -1 in counts else None

# It's day 24
day = 24
lines = get_input_as_lines(day)
ret = d24p1(lines, 65)
print(f'There are {ret} tiles with black side up')

There are 330 tiles with black side up
time: 89.6 ms (started: 2021-02-28 17:09:35 +00:00)


In [ ]:
#@title Part 2 { form-width: "20%" }
import copy

def d24p2(lines, size):
    def count_neighbors(i, j):
        count = 0
        for n in moves.values():
            y = i + n[0]
            x = j + n[1]
            if 0 <= x < hex_grid.shape[1] and 0 <= y < hex_grid.shape[0] \
               and hex_grid[y, x] == -1:
                count += 1
        return count

    def process_day(hex_grid):
        new_grid = copy.deepcopy(hex_grid)
        for (i, j), _ in np.ndenumerate(hex_grid):
            if hex_grid[i, j] == 0:
                continue
            c = count_neighbors(i, j)
            if hex_grid[i, j] == 1 and c == 2:
                new_grid[i, j] *= -1
            if hex_grid[i, j] == -1 and (c == 0 or c > 2):
                new_grid[i, j] *= -1
        
        return new_grid

    hex_grid = process_grid(lines, size)    

    for a in range(100):
      hex_grid = process_day(hex_grid)

    counts = collections.Counter(hex_grid.ravel())
    return counts[-1] if -1 in counts else None


# It's day 24
day = 24
lines = get_input_as_lines(day)
ret = d24p2(lines, 240)
print(f'There are {ret} tiles with black side up')

There are 3711 tiles with black side up
time: 21.5 s (started: 2021-02-28 17:16:11 +00:00)


# Day 25

In [ ]:
#@title Part 1 { form-width: "20%" }
def d25p1(lines):
  card, door = list(map(int, lines))
  card_loops, door_loops = 1, 1
  while pow(7, card_loops, 20201227) != card:
    card_loops += 1
  while pow(7, door_loops, 20201227) != door:
    door_loops += 1
  
  return pow(door, card_loops, 20201227)

# It's day 25
day = 25
lines = get_input_as_lines(day)
ret = d25p1(lines)
print(f'The encryption key is: {ret}')

The encryption key is: 12181021
time: 52 s (started: 2021-03-01 20:26:31 +00:00)
